Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [3]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time

In [4]:
# Selecciono el CSV
elemento_a_buscar = "iphone" # para guardar el archivo   <--------
#iphone = int((elemento_a_buscar).split()[1]) # 12, 14 o 15

In [5]:
# Cargo archivo
carpeta = "3_feature_engineering"
fecha = datetime.now().strftime("%Y%m%d")
#fecha = "20250106"
archivo = carpeta + '/' + elemento_a_buscar.split()[0] + '_' + fecha + '.xlsx'
print(archivo)
df_original = pd.read_excel(carpeta + '/' + elemento_a_buscar.split()[0] + '_' + fecha + '.xlsx')
df_original.head()


3_feature_engineering/iphone_20250108.xlsx


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1033637778,20250108,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True
1,1072128027,20250108,14,pro,128,NaN,"599,00",good,iphone 14 pro 128gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-128...,False,False,False
2,1079442450,20250108,14,pro,256,NaN,"699,00",as_good_as_new,iphone 14 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-256...,False,False,False
3,1079824658,20250108,16,pro,512,NaN,"1320,00",as_good_as_new,iphone 16 pro 512gb,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-512...,False,False,False
4,1085459569,20250108,14,pro max,NaN,NaN,"869,00",new,iphone 14 pro max,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,False,False


In [6]:
# Selecciono el sub-dataframe.
# Selecciono:       as_good_as_new 
#                   iphone 15. 
#                   bateria = NaN
# Son 263 apariciones
# estado = new...
df_to_scrap = df_original[#(df_original['estado'] == estado) & 
                          #(df_original['gen'] == iphone) & 
                          (pd.isna(df_original['bateria']))]
#df_to_scrap = df_original
df_to_scrap.reset_index(drop=True, inplace = True)
df_to_scrap.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1033637778,20250108,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True
1,1072128027,20250108,14,pro,128,NaN,"599,00",good,iphone 14 pro 128gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-128...,False,False,False
2,1079442450,20250108,14,pro,256,NaN,"699,00",as_good_as_new,iphone 14 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-256...,False,False,False
3,1079824658,20250108,16,pro,512,NaN,"1320,00",as_good_as_new,iphone 16 pro 512gb,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-512...,False,False,False
4,1085459569,20250108,14,pro max,NaN,NaN,"869,00",new,iphone 14 pro max,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,False,False


In [7]:
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441

In [8]:

# 263 anuncios - 25 min. con sleep de 1
# 5s por anuncio si va lento. Si va bien 1.5s
# Escrapeo objetos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario


def scrap_comments(df):
    """
    Dado un df con una columna 'url de wallapop':
    accede a la URL, 
    busca el comentario y
    lo guarda en la columna 'comentario'
    """
    df_len = len(df)
    df['comentario'] = None
    if 'url' not in df.columns:
        raise ValueError("Error: La columna 'url' no existe en el DataFrame.")
    print("Tiempo estimado de inicio: 25s...")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(df.loc[0]['url']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()  # Quita cookies y carga primera web
    for index in range(len(df)):
        start_time = time.time()  # Marca de tiempo al inicio de la iteración
        print(index, '/', df_len)
        driver.get(df.loc[index]['url']) 
        try:
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], ': \n', elemento_comentario)
        except:
            #element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            df.loc[index, 'comentario'] = "not found"
            print(df.loc[index]['url'], df.loc[index, 'comentario'])

        end_time = time.time()  # Marca de tiempo al final de la iteración
        execution_time = end_time - start_time  # Calcula el tiempo de ejecución
        print(f"Estimated time to finish: {((df_len - index) * execution_time/60):.2f}min")
        print(f"{execution_time:.2f}s")  
    driver.quit()
    return df

#file = "1_datos_raw/" + "20250102_gopro 13_as_good_as_new.csv"
#df_test = pd.read_csv(file)
df_with_comments = scrap_comments(df_to_scrap)

C:\Users\ACER\AppData\Local\Temp\ipykernel_18668\3237086349.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comentario'] = None


Tiempo estimado de inicio: 25s...
0 / 662
https://es.wallapop.com/item/apple-iphone-15-128gb-verde-revisado-1033637778 : 
 
Estimated time to finish: 58.32min
5.29s
1 / 662
https://es.wallapop.com/item/iphone-14-pro-128gb-1072128027 : 
 Detalles del producto
iphone 14 Pro de 128 GB 
No tiene arañazos ni golpes.
Funciona todo perfectamente: táctil, altavoz, micrófono, sonido, faceid, etc...
Todo original
salud de la batería 88%
lo vendo con la caja y con su cable de carga
Estado
Buen estado
Marca
Apple
Modelo
iPhone 14 Pro
Color
Negro
Capacidad de almacenamiento
128 GB
Estimated time to finish: 62.03min
5.63s
2 / 662
https://es.wallapop.com/item/iphone-14-pro-256gb-1079442450 : 
 Detalles del producto
Iphone 14 pro de 256gb en perfecto estado, sigue con apple care vigente, se vende por cambiar al 16 pro.
perfecto estado con funda, protector de camara y cristal templado
Estado
Como nuevo
Marca
Apple
Modelo
iPhone 14 Pro
Color
Morado
Capacidad de almacenamiento
256 GB
Estimated time to fi

In [9]:
df_with_comments.head(4)


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637778,20250108,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True,
1,1072128027,20250108,14,pro,128,NaN,"599,00",good,iphone 14 pro 128gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-128...,False,False,False,Detalles del producto\niphone 14 Pro de 128 GB...
2,1079442450,20250108,14,pro,256,NaN,"699,00",as_good_as_new,iphone 14 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-256...,False,False,False,Detalles del producto\nIphone 14 pro de 256gb ...
3,1079824658,20250108,16,pro,512,NaN,"1320,00",as_good_as_new,iphone 16 pro 512gb,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-512...,False,False,False,Detalles del producto\nVendo iphone 16 pro pra...


In [10]:
subdf_comentarios = df_with_comments[['id', 'comentario']]
subdf_comentarios

,id,comentario
0,1033637778,
1,1072128027,Detalles del producto\niphone 14 Pro de 128 GB...
2,1079442450,Detalles del producto\nIphone 14 pro de 256gb ...
3,1079824658,Detalles del producto\nVendo iphone 16 pro pra...
4,1085459569,Detalles del producto\nOFERTA.\nApple iPhone 1...
...,...,...
657,1081325330,Apple\nIPHONE 14 PRO 256GB\nSILVER\n\nTeléfono...
658,1081325332,Apple\nIPHONE 14 PRO 256GB\nPURPLE\n\nTeléfono...
659,1081325326,Apple\nIPHONE 14 PRO 256GB\nGOLD\n\nTeléfono r...
660,1081318832,Apple\nIPHONE 14 PRO 256GB\nPURPLE\n\nTeléfono...


In [11]:
# Guardo comentarios finales en el df original
df_original = pd.merge(df_original, subdf_comentarios, on='id', how='left')


In [12]:
df_original.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637778,20250108,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True,
1,1072128027,20250108,14,pro,128,NaN,"599,00",good,iphone 14 pro 128gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-128...,False,False,False,Detalles del producto\niphone 14 Pro de 128 GB...
2,1079442450,20250108,14,pro,256,NaN,"699,00",as_good_as_new,iphone 14 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-256...,False,False,False,Detalles del producto\nIphone 14 pro de 256gb ...
3,1079824658,20250108,16,pro,512,NaN,"1320,00",as_good_as_new,iphone 16 pro 512gb,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-512...,False,False,False,Detalles del producto\nVendo iphone 16 pro pra...
4,1085459569,20250108,14,pro max,NaN,NaN,"869,00",new,iphone 14 pro max,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,False,False,Detalles del producto\nOFERTA.\nApple iPhone 1...


In [13]:
nombre_archivo_csv = df_original
carpeta = "4_download_description"
nombre_archivo_pkl = fecha + '_' + elemento_a_buscar
df_original.to_csv(carpeta + '/' + nombre_archivo_pkl + '.csv')